# Рекомендация тарифов

В вашем распоряжении данные о поведении клиентов, которые уже перешли на эти тарифы (из проекта курса «Статистический анализ данных»). Нужно построить модель для задачи классификации, которая выберет подходящий тариф. Предобработка данных не понадобится — вы её уже сделали.

Постройте модель с максимально большим значением *accuracy*. Чтобы сдать проект успешно, нужно довести долю правильных ответов по крайней мере до 0.75. Проверьте *accuracy* на тестовой выборке самостоятельно.

In [1]:
import pandas as pd

from sklearn.model_selection import train_test_split

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

from sklearn.metrics import accuracy_score

from sklearn.dummy import DummyClassifier

## Изучение данных

In [2]:
df = pd.read_csv('/datasets/users_behavior.csv')

In [3]:
display(df.head(5))

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
calls       3214 non-null float64
minutes     3214 non-null float64
messages    3214 non-null float64
mb_used     3214 non-null float64
is_ultra    3214 non-null int64
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


## Разделение на выборки

Разделим данные на обучающую, валидационную и тестовую выборки размером 60%, 20% и 20%.

In [5]:
df_train, df_valid = train_test_split(df, test_size=0.4, random_state=12345)

df_valid, df_test = train_test_split(df_valid, test_size=0.5, random_state=12345)

In [6]:
print("Обучающая выборка состоит из {} объектов, доля - {:.0f}%".format(df_train.shape[0], df_train.shape[0]/df.shape[0]*100))
print("Валидационная выборка состоит из {} объектов, доля - {:.0f}%".format(df_valid.shape[0], df_valid.shape[0]/df.shape[0]*100))
print("Тестовая выборка состоит из {} объектов, доля - {:.0f}%".format(df_test.shape[0], df_test.shape[0]/df.shape[0]*100))

Обучающая выборка состоит из 1928 объектов, доля - 60%
Валидационная выборка состоит из 643 объектов, доля - 20%
Тестовая выборка состоит из 643 объектов, доля - 20%


## Исследование моделей

In [7]:
#Валидационная выборка
valid_features = df_valid.drop(['is_ultra'], axis=1)
valid_target = df_valid['is_ultra']

#Обучающая выборка
train_features = df_train.drop(['is_ultra'], axis=1)
train_target = df_train['is_ultra']

In [8]:
# Функция расчитывающая "оценку правильности" на выборке
def get_accuracy_score(model, m_type):
    model.fit(train_features, train_target)
    
    if m_type == "valid":
        predictions = model.predict(valid_features)
        score = accuracy_score(valid_target, predictions)
    
    elif m_type == "test":
        predictions = model.predict(test_features)
        score = accuracy_score(test_target, predictions)
        
    elif m_type == "dummy":
        score = model.score(valid_features, valid_target)
        
    return score

In [9]:
# Функция обучающая модель по заданному алгоритму и возвращающая лучшую "оценку правильности" и глубину дерева
def model_train(algorithm_type):
    best_result = 0
    best_depth = 0
    best_estim = 0
    estim = 0
    
    for depth in range(1, 11):
        if algorithm_type == "DecisionTreeClassifier":
            model = DecisionTreeClassifier(max_depth=depth, random_state=12345)
            
            result = get_accuracy_score(model, "valid")
        
        elif algorithm_type == "RandomForestClassifier":
            for estim in range(1, 11):
                model = RandomForestClassifier(n_estimators=estim, max_depth=depth, random_state=12345)
                result = get_accuracy_score(model, "valid")
                
        elif algorithm_type == "LogisticRegression":
            model = LogisticRegression(random_state=12345)
            best_result = get_accuracy_score(model, "valid")
            
            break

        if result > best_result:
            best_depth = depth
            best_estim = estim
                
            best_result = result
    
    if best_estim > 0 and best_depth > 0:
        print('best accuracy = {}, best estim = {}, best depth = {}'.format(best_result, best_estim, best_depth))
        
    elif best_depth > 0:
        print('best accuracy = {}, best depth = {}'.format(best_result, best_depth))
        
    else:
        print('best accuracy = {}'.format(best_result))

### Дерево решений

In [10]:
model_train("DecisionTreeClassifier")

best accuracy = 0.7853810264385692, best depth = 3


### Случайный лес

In [11]:
model_train("RandomForestClassifier")

best accuracy = 0.8009331259720062, best estim = 10, best depth = 6


### Логистическая регрессия

In [12]:
model_train("LogisticRegression")

best accuracy = 0.7589424572317263


/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


**Вывод**

Лучший результат у алгоритма RandomForestClassifier.

## Проверка модели на тестовой выборке

In [13]:
#Тестовая выборка
test_features = df_test.drop(['is_ultra'], axis=1)
test_target = df_test['is_ultra']

Зададим лучшие параметры полученные ранее.

In [14]:
test_model = RandomForestClassifier(n_estimators=10, max_depth=6, random_state=12345)

test_result = get_accuracy_score(test_model, "test")

print("Accuracy:", test_result)

Accuracy: 0.7916018662519441


## Проверка модели на адекватность

In [15]:
dc_model = DummyClassifier(strategy = "most_frequent", random_state = 12345)

dc_result = get_accuracy_score(dc_model, "dummy")

print("Accuracy:", dc_result)

Accuracy: 0.7060653188180405


**Общий вывод**

Мы сравнили несколько алгоритмов (DecisionTreeClassifier, RandomForestClassifier, LogisticRegression) и подобрали для них оптимальные гиперпараметры. Самым успешным можно считать RandomForestClassifier с 81% успешных предсказаний. Проверка на тестовой выборке показала тоже хорошие результаты - 79%. Модель можно считать адекватной!